## 4. (OPTIONAL) NetCDFS and XArray  <a name='bookmark4' />

The following is cool to follow along and see what can be done, but it is a lot! We encourage reading through and running the cells to see what happens, but don't stress too much if you aren't completely following. For Bio students, it may be a bit more confusing because we aren't as familiar with the data - that's okay, the resulting images are pretty neat. :D

If you want to keep going...so when Profs Spera and Yang were mere grad students, when Vine was the original TikTok, the iPhone 5s was released, and Thrift Shope by Macklemore and Ryan was unavoidable, numpy was the way big datasets were dealt with. But now, over a decade later, there's a fancier, newer package, called `xarray` - which is essentially built upon `numpy` - and it is quite the treat for dealing with netcdfs and geotiffs, and basically any gridded dataset where each pixel value also has a lat/lon value. 

In [ ]:
# lets import some packages
import xarray as xr
import pandas as pd

import matplotlib.pyplot as plt

Because large gridded dataset files are huge, let's just download them directly. We can use a for loop to download more than one file, but let's work with one for right now. We'll be using NCEP's North American Regional Reanalysis (NARR) at the daily height of the planetary boundary layer (pbl) in 2023. More info here. The pbs is the lowest part of the atmosphere where winds are influened by friction and it's height is important for things like weather forecasting and air travel.

When you run the cell below, you'll likely get a SerializationWarning. This warning is simply saying that Xarray is reading all instances of ± 9.96921e+36 as not a number (nan). We can check the validity of this on the NCEP info page. On the right hand side under Missing Data, missing values are indeed replaced with ± 9.96921e+36.

In [ ]:
# link to the file in noaa's repository
file = 'https://psl.noaa.gov/thredds/fileServer/Datasets/NARR/Dailies/monolevel/hpbl.2024.nc'
# use xarray to open the file
data = xr.open_dataset('https://psl.noaa.gov/thredds/fileServer/Datasets/NARR/Dailies/monolevel/hpbl.2024.nc')

# print the data
# this might take a hot sec.
#data

Okay, fun! It doesn't look like a spreadsheet. Let's break this down.

**Array Type**: The very first thing that is shown in the above print out is 'xarray.Dataset'. There are two main types of arrays that xarray can handle: DataArrays and Datasets. A Dataset is a collection of DataArrays. Imagine that a DataArray is a rubix cube. It is a 3D (or 2D or 4D or however shaped) array of data. A Dataset would therefore be a collection of rubix cubes. Imagine you ordered a shipment of 30 rubix cubes. Each individual rubix cube is a DataArray, and the box that all the cubes were shipped in is the Dataset. We can select one DataArray using its variable name. There are some data selection methods that only work on DataArrays, which is why it is important to distinguish between DataArrays and Datasets.

**Dimensions**: The dimensions are the directions that the file has, or the names of the axes. There are four dimensions listed in this file: `time`, `x`, `y`, and `nbnds`. Since this is meteorological data, we can think of these as latitudes or rows and longitudes or columns. There are 277 y, which means there are 277 lines of latitude in the grid. There are 349 x, or 349 lines of longitude. The time dimension has 365 values. We selected daily means over a year. If we had selected data from 2020, the number would be 366 because 2020 was a leap year.

**Coordinates**: Coordinates are labels for each step of a dimension. This particular file has three of the same coordinates as in dimensions: `x`, `y`, and `time`. However, `nbnds` is not in the coordinate list, and now there are `lat` and `lon` coordinates. Next to each coordinate is the number and name of the dimensions in that coord. Time, x, and y only have one dimension and those dimensions have the same name. Click on the piece of paper next to each of these coordinates on the right. What does it say? In comparison, `lat` and `lon` have two dimensions: (y,x). This is because any point on a map will have a unique latitude and longitude combination. Each grid point in this Dataset will also have a unique latitude and longitude combination, which is made up of x and y values.

**Data Variables**: These are the variables of interest. This is the actual data in your file. For example, there is `hpbl` that might be of interest to us. In some Datasets, the data variables have unusual names, which can make understanding the data difficult. Xarray is great because it allows you to have metadata for each data variable, which might help to explain the data a little more. Click on the picture of a piece of paper next to the `hpbl` variable name. What does it say? Some of the data variables have useful metadata under this piece of paper.

**Indexes**: Indexes are not always in an Xarray Dataset. Dimensions and coordinates are more useful in Xarray than the indexes, but you might find Datasets that include them. In this case, it is the three dimensions of the data (x,y, time) with a pandas series of the values for each dimension that is listed in the coords.

**Attributes**: Finally, Xarray has more metadata stored in the Attributes. Some of this information can be helpful, such as the lat and lon corners. Knowing the shape and map projection is important for properly formatting the data for visualization.

If you are interested in knowing just the attributes or just the coordinates or just the dims of an Xarray DataArray, you can call this information specifically.

In [ ]:
# get the dimensions of dataset
data.dims

In [ ]:
# get the coordinates of the dataset
data.coords

In [ ]:
# get the attributes of the dataset
data.attrs

In [ ]:
# get the variables in the dataset
data.var

### 4.1. Selecting and indexing data
Earlier we said that DataArrays and Datasets are different. Indexing and selecting data is one of the places that distinguish DataArrays and Datasets. Right now, we have a Dataset. This is a collection of DataArrays. We can select one of these DataArrays from the whole Dataset using the variable name.

In [ ]:
# get a data array from our datset
var = data['hpbl']

# print the data
var

Now that we have a DataArray, we can select the dimension of the data using the positions OR using names, so we can use either an integer or label. We'll try some examples.

In terms of dimensions, the DataArray is set up `[time, x, y]`.

In [ ]:
# take the first time slice but all the values in the x and y dimension
var[0,:,:]

We can also use `isel` (integer select) to get the same result.

In [ ]:
# take the first time slice using names & integers
var.isel(time=0)

We can also use dictionaries and keys.

In [ ]:
var[dict(time=0)]

So, there are multiple ways to select & index data. You can only do this with DataArrays. Datasets are more complicated, and can only be indxed using dimension names, not positions.

We can also slice our data. 
A slice is what it sounds like: it is a piece of the data that we are cutting out from the whole. `slice(None, 3)` says "Cut a piece of this data, starting at the 0th position (None) and ending at the 3rd position (3)." We used the dimension name "y" to tell xarray which dimension to index on. So `y=slice(None,3)` is saying "Cut a piece of the data along the y dimension, starting at the 0th position and ending at the 3rd position." This is how we cut out the first 3 y's of the data.
In the cell below, we print out the dimensions of the original data and of some sliced data. Do the dimensions printed match your expectations?

In [ ]:
print(data.dims)
print(data[dict(y=slice(None,30))].dims)
print(data[dict(x=slice(25,100))].dims)

### 4.2. Net CDF Data Viz

These data are the height of the planetary boundary layer in meters above the surface. The dimensions of the data are x, y, and time. We could plot this data as rows (latitudes) and columns (longitudes) using a grid of 349 by 277 or we could make line plots of average temperatures over one of these dimensions.

What if we wanted to know the average temperature along a latitude or longitude? Perhaps we have the hypothesis that PBL height is higher at lower latitudes closer to the equator. How would we test this hypothesis? We could take the mean of the data along the axis of interest.

Since the PBL data is formatted in a 3D array, and we want to take the average along two of these dimensions to make a line plot of height versus dimension. If we want to know the average at each latitude, then we must take the mean along the longitude and time dimensions. If this is hard to visualize, think back to a rubix cube. We want to know the average at each point along the y-axis (the rows). In order to do this, we must smush the x-axis, so take the average of the columns.

Once again, we can using different indexing methods in Xarray to accomplish the same task. Now let's plot the averages, this time using Xarray plotting.

In [ ]:
# select the variableb
hpbl = data['hpbl']

hpbl

In [ ]:
# find the mean of the data over time
hpbl_timeavg = hpbl.mean(axis=0)
hpbl_timeavg.dims

In [ ]:
# find the mean of the rows/latitudes using dimension names
hpbl_lat = hpbl_timeavg.mean('x')
hpbl_lat.dims

In [ ]:
fig = plt.figure(figsize = (10,6))
hpbl_lat.plot()
plt.show()

In [ ]:
# xarray will do it's best to label the axes 
# but we can make them better

fig = plt.figure(figsize=(10,6))

degrees = 1 + hpbl_lat['y']/111139 # convert x to degrees latitude
plt.plot(degrees,hpbl_lat)
plt.title('Average Planetary Boundary Layer Height')
# you can change the label data for your data

plt.xlabel('Latitude (˚)',fontsize=12)
plt.ylim((0,1000))
plt.ylabel('Height (m)',fontsize=12)
plt.show()

The planetary boundary layer height is higher at lower latitudes - in other words, the atmosphere is thicker closer to the equator. Why might that be? 

Also, what if we want to plot this gridded data on a map? We. Can. Do. It.
You can find a bunch of fun data [here](https://www.ncei.noaa.gov/access/world-ocean-atlas-2018/) but let's look at sea surface temperature data from the summers between 2005-2017 from Argo float [buoys](https://argo.ucsd.edu/). 

In [ ]:
import xarray as xr 
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
from cartopy import crs, feature


In [ ]:
# link to the data
file = 'https://www.ncei.noaa.gov/thredds-ocean/dodsC/ncei/woa/temperature/A5B7/1.00/woa18_A5B7_t15_01.nc'

# why do we have to set decode_times = false here?
# what happens if we don't?
data = xr.open_dataset(file, engine='netcdf4', decode_times=False)

data

Let's select the data and then make a quick and dirty figure.

In [ ]:
temp = data['t_an']
temp


In [ ]:
# basic contour plot using lat, lons, for 
# the first time stamp at the surface (depth = 0)
plt.contourf(temp['lon'], temp['lat'], temp[0,0,:,:])
plt.show()

Now that we've looked at the entire dataset area, we can zoom in on a particular section. There are a few ways to do this, but we'll practice the easiest first.

The easiest method to zoom into an area on a map is to only plot that area with cartopy. This is fast and useful for making a map for presentations, but there are some downsides.

* First, this method does not remove the rest of the data, it just hides it from our view. If you wanted to find the average chlorophyll-a concentration in the zoomed in area, you could not calculate it from the map because the rest of the data actually still exists.
* Second, outliers in the data outside the zoomed in area might affect the colorbar scale of the displayed map. So you could have a map area with concentrations all below 20 mg/m 3, but a larger concentration elsewhere in the dataset will skew the colorbar scale and you won't be able to see the low concentrations clearly.
* Finally, for really big datasets, it might take a while to plot. You're still technically plotting all the data, it just isn't displayed in the figure area. So if the dataset is really big, you're wasting time plotting data that you don't actually see.

In [ ]:
# Let's choose an area of interest
latmin = 37
latmax = 45
lonmin = -65
lonmax = -75

# set extent
extent = [lonmin, lonmax, latmin, latmax]

# set the projection - make sure it matches that of the dataset
proj = crs.PlateCarree()
print(proj)

In [ ]:
# We mentioned this above w. the point data, but with
# gridded data, you have to make a grid of the coordinates

lats = temp['lat']
lons = temp['lon']

# make the gred
XX,YY = np.meshgrid(lons,lats)

In [ ]:
# create a pyplot figure
fig = plt.figure(figsize=(8,10))
# create a new axes instance with the map information
ax = fig.add_subplot(1,1,1,projection=proj)
ax.set_extent(extent)
# add gridlines
gl = ax.gridlines(crs.PlateCarree(),draw_labels=True,linewidth=1,color='gray',alpha=0.5,linestyle='--')
# add features
ax.add_feature(feature.STATES)
ax.add_feature(feature.RIVERS)
im = plt.contourf(XX,YY,temp[0,0,:,:],cmap='turbo',transform=proj)
plt.colorbar(im,orientation='horizontal',label='Temperature (˚C)')
plt.show()

What do you notice about this figure? What about the colorbar scale? What could make this figure better?

The figure above has a colorbar scale that goes from -5 to 40 ˚C, even though the data in the figure does not seem to go this high or low. This is one of the problems with this plotting method. Your colorbar scale may be different if you used a different date of data.

Instead of plotting all the data but hiding most of it, we can instead select a subsection of the data using slicing. This method takes a few more steps because you need to know the area of interest. However, there are several benefits such as being able to really focus on the actual area of interest.

In [ ]:
# slice the data using these limits
subset = temp.sel(lat=slice(latmin,latmax),lon=slice(lonmax,lonmin))

# check out the data and the change in dimensions
subset

In [ ]:
# in order to plot a subset of the data on the map, 
# we must make a grid of the coordinates
# first, get a list of each coordinate
lats = subset['lat']
lons = subset['lon']

# then make a grid
XX,YY = np.meshgrid(lons,lats)

In [ ]:
# create a pyplot figure
fig = plt.figure(figsize=(8,10))
# create a new axes instance with the map information
ax = fig.add_subplot(1,1,1,projection=proj)
ax.set_extent(extent)
# add gridlines
gl = ax.gridlines(crs.PlateCarree(),draw_labels=True,linewidth=1,color='gray',alpha=0.5,linestyle='--')
# add features
ax.add_feature(feature.STATES)
ax.add_feature(feature.RIVERS)
im = plt.contourf(XX,YY,subset[0,0,:,:],cmap='turbo',transform=proj)
plt.colorbar(im,orientation='horizontal',label='Temperature (˚C)')
plt.show()

The code ran faster because we sliced the data. 
But, we still have giant empty spaces because the data is at a 1 deg by 1 deg (100 km by 100 km) spatial resolution.

If you want to try one more plot you can - if not, no big! Here we'll use annual average density on a quarter degree (1/4 deg ~ 25 km) grid).

In [ ]:
file2 = 'https://www.ncei.noaa.gov/thredds-ocean/dodsC/ncei/woa/density/decav/0.25/woa18_decav_I00_04.nc'

data2 = xr.open_dataset(file2,engine='netcdf4',decode_times=False)

data2

In [ ]:
# get the data we want
dens = data2['I_an'][0,0,:,:]
dens

In [ ]:
# get our lats and lons again
latmin = 37
latmax = 45
lonmin = -65
lonmax = -75

# get the subset
subset = dens.sel(lat=slice(latmin,latmax),lon=slice(lonmax,lonmin))

# set the extent of our box
extent = [lonmin,lonmax,latmin,latmax]

# set the right map projection
proj = crs.PlateCarree()

In [ ]:
# in order to plot a subset of the data on the map, we must make a grid of the coordinates
# first, get a list of each coordinate
lats = subset['lat']
lons = subset['lon']

# then make a grid
XX,YY = np.meshgrid(lons,lats)

In [ ]:
# create a pyplot figure
fig = plt.figure(figsize=(8,10))
# create a new axes instance with the map information
ax = fig.add_subplot(1,1,1,projection=proj)
ax.set_extent(extent)
# add gridlines
gl = ax.gridlines(crs.PlateCarree(),draw_labels=True,linewidth=1,color='gray',alpha=0.5,linestyle='--')
# add features
ax.add_feature(feature.STATES)
ax.add_feature(feature.RIVERS)
im = plt.contourf(XX,YY,subset,cmap='BuPu',transform=proj)
plt.colorbar(im,orientation='horizontal',label='Density $(kg/m^{3})$')
plt.show()

Let's zoom out! 

In [ ]:
lats = dens['lat']
lons = dens['lon']

XX,YY = np.meshgrid(lons,lats)

In [ ]:
# create a pyplot figure
fig = plt.figure(figsize=(8,10))
# create a new axes instance with the map information
ax = fig.add_subplot(1,1,1,projection=proj)
ax.set_extent([-180,180,-90,90])
# add gridlines
gl = ax.gridlines(crs.PlateCarree(),draw_labels=True,linewidth=1,color='gray',alpha=0.5,linestyle='--')
# add features
ax.add_feature(feature.LAND,color='grey',alpha=0.3)
ax.add_feature(feature.RIVERS)
im = plt.contourf(XX,YY,dens,cmap='BuPu',transform=proj)
plt.colorbar(im,orientation='horizontal',label='Density $(kg/m^{3})$')
plt.show()

It appears that here things have a density of 0, which is impossible? Let's see what the minimum density is.

In [ ]:
dens.min()

Okay, at least the minimum isn't actually zero, but it is really small. This highlights the importance of knowing your data. These density values we've been using are not absolute densities, they are sigma-t densities. They are the difference from  1000kg/m3, which is why we can have such small values. In reality, the density values should be 1000 + dens.

In [ ]:
# create a pyplot figure
fig = plt.figure(figsize=(8,10))
# create a new axes instance with the map information
ax = fig.add_subplot(1,1,1,projection=proj)
ax.set_extent([-180,180,-90,90])
# add gridlines
gl = ax.gridlines(crs.PlateCarree(),draw_labels=True,linewidth=1,color='gray',alpha=0.5,linestyle='--')
# add features
ax.add_feature(feature.LAND,color='grey',alpha=0.3)
ax.add_feature(feature.RIVERS)
im = plt.contourf(XX,YY,dens+1000,cmap='BuPu',transform=proj)
plt.colorbar(im,orientation='horizontal',label='Density $(kg/m^{3})$')
plt.show()

We done.

<a href=#home>Return to Top</a> 